In [1]:
import numpy as np
import pandas as pd

In [2]:
incident = pd.read_csv("/content/California_Fire_Incidents.csv")

In [3]:
interested = ['Started','Counties','AcresBurned','Injuries','Fatalities','StructuresDestroyed','StructuresDamaged',]

In [4]:
incident_data = incident[interested].copy()

In [5]:
county = ['Tuolumne', 'Los Angeles', 'Riverside', 'Placer', 'Ventura',
       'Fresno', 'Siskiyou', 'Humboldt', 'Tehama', 'Shasta', 'San Diego',
       'Kern', 'Sonoma', 'Contra Costa', 'Butte', 'Tulare',
       'Santa Barbara', 'Mariposa', 'Monterey', 'El Dorado',
       'San Bernardino', 'Plumas', 'Modoc', 'San Luis Obispo', 'Madera',
       'Inyo', 'Napa', 'San Benito', 'San Joaquin', 'Lake', 'Alameda',
       'Glenn', 'Yolo', 'Sacramento', 'Stanislaus', 'Solano', 'Merced',
       'Mendocino', 'Lassen', 'Amador', 'Yuba', 'Nevada', 'Santa Clara',
       'Calaveras', 'San Mateo', 'Orange', 'Colusa', 'Trinity',
       'Del Norte', 'Mono', 'Alpine', 'Sutter', 'Kings', 'Sierra',
       'Santa Cruz', 'Marin']

In [6]:
incident_data = incident_data[incident_data['Counties'].isin(county )].copy()

In [7]:
tmp = []
for i in incident_data["Started"]:
  date = i.split('-')
  tmp.append(date[0] + date[1])
incident_data["Started"] = tmp

In [8]:
incident_data = incident_data.rename(columns={'Counties': 'County'})

In [9]:
county = []
for i in incident_data["County"]:
  name = i.split(" ")
  tmp = "".join([i.lower() for i in name])
  county.append(tmp.capitalize())
incident_data["County"] = county

In [10]:
incident_data = incident_data.fillna(0)

In [11]:
group = dict()
for i in range(len(incident_data)):
  if incident_data.iloc[i]['Started'] != '196912':
    key = incident_data.iloc[i]['Started'] + incident_data.iloc[i]['County']
    if key not in group:
      group[key] = incident_data.iloc[i].to_dict()
    else:
      group[key]['AcresBurned'] += incident_data.iloc[i]['AcresBurned']
      group[key]['Injuries'] += incident_data.iloc[i]['Injuries']
      group[key]['Fatalities'] += incident_data.iloc[i]['Fatalities']
      group[key]['StructuresDestroyed'] += incident_data.iloc[i]['StructuresDestroyed']
      group[key]['StructuresDamaged'] += incident_data.iloc[i]['StructuresDamaged']

In [12]:
tmp = []
for k,v in group.items():
  tmp.append(v)
combo_incident_data = pd.DataFrame(tmp)

In [13]:
combo_incident_data

,Started,County,AcresBurned,Injuries,Fatalities,StructuresDestroyed,StructuresDamaged
0,201308,Tuolumne,258384.0,0.0,0.0,0.0,0.0
1,201305,Losangeles,31135.0,0.0,0.0,0.0,0.0
2,201307,Riverside,27736.0,0.0,0.0,0.0,0.0
3,201308,Placer,27440.0,0.0,0.0,0.0,0.0
4,201305,Ventura,28597.0,14.0,0.0,10.0,6.0
...,...,...,...,...,...,...,...
903,201909,Amador,10.0,1.0,0.0,0.0,0.0
904,201907,Butte,10.0,0.0,0.0,0.0,0.0
905,201904,Butte,10.0,0.0,0.0,0.0,0.0
906,201906,Nevada,2.0,0.0,0.0,0.0,0.0


In [14]:
property_value = pd.read_csv("/content/Housing Price.csv")

In [15]:
property_value = property_value.fillna("0")

In [16]:
for i in property_value.columns:
  if i != 'Mon-Yr':
    property_value[i] = property_value[i].str.replace('$', '', ).str.replace(',', '').astype(int)

In [17]:
property_value.set_index(property_value.columns[0], inplace=True)

In [18]:
for i in property_value.columns:
  mean = sum(property_value[i][property_value[i] != 0]) / sum(property_value[i] != 0)
  property_value[i] = property_value[i].replace(0, mean)

In [19]:
lookup = {
    "Losangeles": "Los Angeles",
    "Sandiego" : "San Diego",
    'Contracosta':'Contra-Costa',
    'Santabarbara': 'Santa Barbara',
    'Eldorado':'El Dorado',
    'Sanbernardino':'San Bernardino',
    'Sanbenito': 'San Benito',
    'Sanjoaquin': 'San Joaquin',
    'Santaclara': 'Santa Clara',
    'Sanmateo': 'San Mateo',
    'Santacruz': 'Santa Cruz'
}

translate_date = {
    '01':"Jan",
    '02':"Feb",
    '03':"Mar",
    '04':"Apr",
    '05':"May",
    '06':"Jun",
    '07':"Jul",
    '08':"Aug",
    '09':"Sep",
    '10':"Oct",
    '11':"Nov",
    '12':"Dec",
}

In [20]:
tmp = []
for i in range(len(combo_incident_data)):
  county = combo_incident_data.iloc[i]['County']
  date = combo_incident_data.iloc[i]['Started']
  year = date[2:4]
  #print(date)
  month = translate_date[date[4:6]]
  fomatted = month + '-' + year
  value = ''
  if county in property_value.columns:
    value = property_value[county][fomatted]
  elif county == "Modoc":
    value = 212000
  elif county == "Sanluisobispo":
    value = 900000
  elif county == 'Inyo':
    value = 794500
  elif county == 'Colusa':
    value = 525000
  elif county == 'Delnorte':
    value = 366568
  elif county == 'Alpine':
    value = 511189
  elif county == 'Sierra':
    value = 449000
  else:
    value = property_value[lookup[county]][fomatted]

  dmg = value * combo_incident_data.iloc[i]['StructuresDestroyed'] + \
        value * combo_incident_data.iloc[i]['StructuresDamaged'] * 0.5
  tmp.append(dmg)

In [21]:
combo_incident_data['PropetyValue Damage'] = tmp

In [22]:
combo_incident_data

,Started,County,AcresBurned,Injuries,Fatalities,StructuresDestroyed,StructuresDamaged,PropetyValue Damage
0,201308,Tuolumne,258384.0,0.0,0.0,0.0,0.0,0.0
1,201305,Losangeles,31135.0,0.0,0.0,0.0,0.0,0.0
2,201307,Riverside,27736.0,0.0,0.0,0.0,0.0,0.0
3,201308,Placer,27440.0,0.0,0.0,0.0,0.0,0.0
4,201305,Ventura,28597.0,14.0,0.0,10.0,6.0,6459310.0
...,...,...,...,...,...,...,...,...
903,201909,Amador,10.0,1.0,0.0,0.0,0.0,0.0
904,201907,Butte,10.0,0.0,0.0,0.0,0.0,0.0
905,201904,Butte,10.0,0.0,0.0,0.0,0.0,0.0
906,201906,Nevada,2.0,0.0,0.0,0.0,0.0,0.0


In [23]:
import json

In [24]:
with open('/content/drought_county.json', 'r') as file:
    drought = json.load(file)

In [25]:
weather_county = drought.keys()

In [26]:
weather_county

dict_keys(['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa', 'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino', 'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito', 'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin', 'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara', 'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus', 'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo', 'Yuba'])

In [27]:
lookup_weather = {
    "Losangeles": "Los Angeles",
    "Sandiego" : "San Diego",
    'Contracosta':'Contra Costa',
    'Santabarbara': 'Santa Barbara',
    'Eldorado':'El Dorado',
    'Sanbernardino':'San Bernardino',
    'Sanbenito': 'San Benito',
    'Sanjoaquin': 'San Joaquin',
    'Santaclara': 'Santa Clara',
    'Sanmateo': 'San Mateo',
    'Santacruz': 'Santa Cruz',
    'Sanluisobispo': 'San Luis Obispo',
    'Delnorte': 'Del Norte'
}

In [28]:
tmp = []
for i in range(len(combo_incident_data)):
  date = combo_incident_data.iloc[i]['Started']
  county = combo_incident_data.iloc[i]['County']
  if county in weather_county:
    tmp.append(drought[county][date]['value'])
  else:
    tmp.append(drought[lookup_weather[county]][date]['value'])
combo_incident_data['Drought Index'] = tmp

In [29]:
with open('/content/precipitation_county.json', 'r') as file:
    precipitation = json.load(file)

In [30]:
tmp = []
for i in range(len(combo_incident_data)):
  date = combo_incident_data.iloc[i]['Started']
  county = combo_incident_data.iloc[i]['County']
  if county in weather_county:
    tmp.append(precipitation[county][date]['value'])
  else:
    tmp.append(precipitation[lookup_weather[county]][date]['value'])
combo_incident_data['Precipitation'] = tmp

In [31]:
with open('/content/average_temperature_county.json', 'r') as file:
    temperature = json.load(file)

In [32]:
tmp = []
for i in range(len(combo_incident_data)):
  date = combo_incident_data.iloc[i]['Started']
  county = combo_incident_data.iloc[i]['County']
  if county in weather_county:
    tmp.append(temperature[county][date]['value'])
  else:
    tmp.append(temperature[lookup_weather[county]][date]['value'])
combo_incident_data['Temperature'] = tmp

In [33]:
with open('/content/heating_county.json', 'r') as file:
    heating = json.load(file)

In [34]:
tmp = []
for i in range(len(combo_incident_data)):
  date = combo_incident_data.iloc[i]['Started']
  county = combo_incident_data.iloc[i]['County']
  if county in weather_county:
    tmp.append(heating[county][date]['value'])
  else:
    tmp.append(heating[lookup_weather[county]][date]['value'])
combo_incident_data['Heating Degree Days'] = tmp

In [35]:
with open('/content/cooling_county.json', 'r') as file:
    cooling = json.load(file)

In [36]:
tmp = []
for i in range(len(combo_incident_data)):
  date = combo_incident_data.iloc[i]['Started']
  county = combo_incident_data.iloc[i]['County']
  if county in weather_county:
    tmp.append(cooling[county][date]['value'])
  else:
    tmp.append(cooling[lookup_weather[county]][date]['value'])
combo_incident_data['Cooling Degree Days'] = tmp

In [37]:
check = set()
for i in range(len(combo_incident_data)):
  check.add(combo_incident_data.iloc[i]['Started'] + combo_incident_data.iloc[i]['County'])

In [38]:
len(check)

908

In [39]:
len(combo_incident_data)

908

In [40]:
combo_incident_data.to_csv('History_County_Weather.csv', index=False)